In [1]:
!pip install gradio requests python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00


In [7]:
import os
import glob
import requests
from dotenv import load_dotenv
import gradio as gr
from google.colab import files
from google.colab import userdata

In [3]:
os.makedirs("knowledge-base/employees", exist_ok=True)

In [6]:
uploaded = files.upload()
for filename in uploaded.keys():
    with open(f"knowledge-base/employees/{filename}", "wb") as f:
        f.write(uploaded[filename])

Saving Emily Carter.md to Emily Carter.md


In [8]:
api_key = userdata.get("GROQ_API_KEY")

In [9]:
endpoint = "https://api.groq.com/openai/v1/chat/completions"
model_id = 'llama3-70b-8192'
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [10]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up."

In [13]:
context = {}
employees = glob.glob("knowledge-base/employees/*.md")
for employee in employees:
    name = os.path.basename(employee)[:-3]  # Remove .txt extension
    with open(employee, "r", encoding="utf-8") as f:
        context[name] = f.read()

In [31]:
def get_relevant_context(message: str) -> list:
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context if relevant_context else ["I dont have enough information on that topic"]

In [32]:
def add_context(message: str):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [33]:
def chat(message: str, history: list):
    messages = [{"role": "system", "content": system_message}] + history
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    payload = {
        "model": model_id,
        "messages": messages
    }

    response = requests.post(endpoint, headers=headers, json=payload, verify=False)

    if response.status_code == 200:
        data = response.json()
        return data["choices"][0]["message"]["content"]
    else:
        return f"Error {response.status_code}: {response.text}"

In [34]:
history = []
user_message = "Who is David?"
response = chat(user_message, history)
print("Response:", response)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.groq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response: I'm not aware of a specific "David" associated with Insurellm. There might be multiple individuals with the name David working at or associated with the company, but without more context, I cannot provide a specific answer.


In [35]:
gr.ChatInterface(fn=chat).launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2f2521ba8deb027cce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
